Первое, что нужно сделать — это загрузить данные, чтобы увидеть, что мы имеем для проведения анализа, на основании каких данных можем сделать градацию каждого признака.

1. Открываем Anaconda, заходим в Jupyter.
2. Добавляем скачанную таблицу в папку для анализа.
3. Создаем файл python.
4. Подключаем модуль, который позволит работать с данными. Это модуль pandas. Импортируем данные, формируем датафрейм, с которым дальше можно продолжать работу.

In [44]:
import pandas as pd
orders = pd.read_csv('/Users/yanaprynik/Work_doc/RFM.csv', sep=',')

Посмотрим на данные 

In [45]:
orders.head()

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franz�sisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21


Приведём данные к тому формату, с которым можно работать.

In [46]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

Найдём самую позднюю покупку.

In [47]:
orders.max()

order_date     2014-12-31 00:00:00
order_id            US-2014-169551
customer        Zuschuss Donatelli
grand_total                  23661
dtype: object

Создаём таблицу RFM. 

Поскольку новизна вычисляется для определённого момента времени, а дата последнего заказа набора данных Tableau SuperStore — 31 декабря 2014 года. Именно эту дату мы будем использовать для расчёта новизны. Установите эту дату на текущий день и извлеките все заказы до вчерашнего дня.

Установите эту дату на текущий день и извлеките все заказы до вчерашнего дня.

Чтобы создать таблицу, нам нужно проанализировать данные и сгруппировать их. Dataframe имеет метод .groupby(), который группирует данные по одному или нескольким столбцам.

Нам нужно найти все данные по каждому клиенту и их преобразовать особым образом, чтобы подсчитать три показателя RFM-анализа.

In [48]:
import datetime as dt

NOW = dt.datetime(2014,12,31)

In [49]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'order_id': lambda x: len(x),      # Frequency #Количество заказов
                                        'grand_total': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

In [50]:
rfmTable

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249
...,...,...,...
Xylona Preis,43,11,2375
Yana Sorensen,9,8,6721
Yoseph Carroll,4,5,5455


Теперь мы можем сделать разбивку по квантилям. Возьмем четыре параметра: 0,25 0,5 0,75 и 1. В Python имеется встроенная функция, которая позволит выполнить эту задачу:

In [51]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,30.0,5.0,1145.0
0.50,75.0,6.0,2257.0
0.75,183.0,8.0,3784.0


Чтобы нам было удобнее работать дальше с этими данными, их необходимо преобразовать в словарь.

Теперь мы можем приступить к формированию таблицы сегментации:

In [52]:
quantiles = quantiles.to_dict()
quantiles

{'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0},
 'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0}}

In [53]:
rfmSegmentation = rfmTable
rfmSegmentation

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249
...,...,...,...
Xylona Preis,43,11,2375
Yana Sorensen,9,8,6721
Yoseph Carroll,4,5,5455


Создадим вспомогательную таблицу, в которую и будем вносить все дальнейшие изменения.

Здесь важно учесть, что показатель давности покупки будет лучше, если он меньше, поэтому создадим две отдельные функции. Одна функция будет работать с показателем R, другая — с F и M.

In [54]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [55]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

Теперь можем получить итоговое значение:

In [56]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [57]:
rfmSegmentation

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Aaron Bergman,415,3,887,4,4,4,444
Aaron Hawkins,12,7,1744,1,2,3,123
Aaron Smayling,88,7,3050,3,2,2,322
Adam Bellavance,54,8,7756,2,2,1,221
Adam Hart,34,10,3249,2,1,2,212
...,...,...,...,...,...,...,...
Xylona Preis,43,11,2375,2,1,2,212
Yana Sorensen,9,8,6721,1,2,1,121
Yoseph Carroll,4,5,5455,1,4,1,141
